In [236]:
import pandas as pd
import requests
import json
from config import Api_Key

In [239]:
# Create your API key
api_key = Api_Key

In [240]:
#create the url and get the json response of all the states 
url = f"http://api.eia.gov/category/?api_key={api_key}&category_id=238627"
USA_JSON = requests.get(url).json()

# Create 
child = USA_JSON['category']['childseries']

In [241]:
url = f"http://api.eia.gov/category/?api_key={api_key}&category_id=238627"
USA_JSON = requests.get(url).json()

# Set the list of dictionaries that contains the series ID for each state to a variable
child = USA_JSON['category']['childseries']

In [242]:
# Create a for loop to append the states series ID to the annual or montly list 
monthly_series_id = []
annual_series_id = []
for cuban in child:
    if cuban['f'] == 'M':
        monthly_series_id.append(cuban['series_id'])
    elif cuban['f'] == 'A':
        annual_series_id.append(cuban['series_id'])

In [243]:
dict_annual = {}
year = True
for series in annual_series_id:
    url_state = f'https://api.eia.gov/series/?api_key={api_key}&series_id={series}'
    print
    state_request = requests.get(url_state).json()
    data = state_request['series'][0]
    state = data['geography'].split('-')
    for value in data['data']:
            if value[1] != None:
                if year == True:
                    dict_annual.setdefault('Year', []).append(value[0])
                if len(state) == 2:
                    dict_annual.setdefault(f'{state[1]}', []).append(value[1])
    year = False

In [244]:
# Remove DC as it does equal the total array number
dict_annual.pop('DC')

[2.379, 0.981, 1.075]

In [245]:
# Create Dataframe and set index
df_annual = pd.DataFrame(dict_annual).set_index('Year')

In [246]:
dict_monthly = {}
year = True
for series in monthly_series_id:
    url_state = f'https://api.eia.gov/series/?api_key={api_key}&series_id={series}'
    print
    state_request = requests.get(url_state).json()
    data = state_request['series'][0]
    state = data['geography'].split('-')
    for value in data['data']:
        x= list(value[0])
        date = f'{x[4]}{x[5]}-{x[0]}{x[1]}{x[2]}{x[3]}'
        if value[1] != None:
            if year == True:
                dict_monthly.setdefault('Year', []).append(date)
            if len(state) == 2:
                dict_monthly.setdefault(f'{state[1]}', []).append(value[1])
    year = False

In [247]:
df_monthly = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in dict_monthly.items() ])).set_index('Year').drop('DC', axis = 1)

In [248]:
df_monthly.index.rename('Date', inplace = True)

In [230]:
# Export to a CSV 
df_monthly.to_csv("States_Monthly.csv")

In [231]:
# Export to a CSV
df_annual.to_csv("State_Annually.csv")

In [250]:
# DataFrames
df_annual
df_monthly

,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
Date,,,,,,,,,,,,,,,,,,,,,
02-2011,3.287,2.659,2.707,2.774,2.911,2.555,2.815,2.691,2.663,2.682,...,2.759,2.620,2.634,2.591,2.663,2.846,2.691,2.687,2.751,2.533
01-2011,3.190,2.556,2.562,2.562,2.746,2.423,2.738,2.671,2.599,2.578,...,2.646,2.527,2.510,2.319,2.596,2.771,2.586,2.594,2.671,2.412
12-2010,3.167,2.450,2.464,2.446,2.664,2.260,2.663,2.577,2.486,2.469,...,2.533,2.420,2.403,2.307,2.499,2.694,2.515,2.487,2.567,2.349
11-2010,3.129,2.296,2.287,2.347,2.560,2.212,2.540,2.431,2.325,2.304,...,2.435,2.268,2.246,2.321,2.337,2.535,2.445,2.364,2.448,2.320
10-2010,3.112,2.242,2.239,2.276,2.509,2.220,2.412,2.319,2.261,2.243,...,2.379,2.220,2.219,2.352,2.275,2.413,2.403,2.314,2.367,2.337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05-1983,NaN,0.991,1.023,0.950,1.012,0.951,1.001,1.006,0.991,0.997,...,1.002,1.006,0.979,0.937,1.002,1.006,0.854,0.993,1.018,1.019
04-1983,NaN,0.958,NaN,0.903,0.941,0.917,0.941,0.977,0.964,0.968,...,0.959,1.001,0.930,0.892,0.958,NaN,0.806,1.005,1.003,0.999
03-1983,NaN,0.931,NaN,0.961,0.877,0.935,0.975,0.943,0.942,0.937,...,0.931,0.929,0.917,0.855,0.926,NaN,0.853,0.916,0.960,0.942
